# Задание 1.
Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

# Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>


In [5]:
# время полного прохода по 38 страницам около 13 мин (в итоге 749 ссылок)
%%time
import time
import requests as req
import pandas as pd
from bs4 import BeautifulSoup      
# формируем переменную, в которой будут наши искомые слова
KEYWORDS = ['python', 'парсинг']
url = 'https://habr.com/ru/search'
all_links = []
text_list = []
href_list = []
#Формируем параметры для запроса
params = {
    'q' : KEYWORDS,
    'target_type' : 'posts',
    'order' : 'relevance'    
}
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0'
}
res = req.get(url, params = params)
soup = BeautifulSoup(res.text)
# получив суп, ищем ссылки на следующие страницы
pages_blocks = soup.find_all('a', class_= 'tm-pagination__page')
# определяем последнюю страницу, чтобы перебирать все страницы до нее
last_page = int(pages_blocks[-1].text)
all_post = pd.DataFrame()
i = 0
# в цикле перебираем блоки preview и собираем ссылки на полный текст в каждой странице
for num_page in range(1, last_page + 1): 
    url = 'https://habr.com/ru/search/page' + str(num_page)
    res = req.get(url, params = params)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text)
    div_blocks = soup.find_all('div', class_= 'tm-article-snippet')
    # иногда попадаются блоки где нет перехода на полный текст, в таком случае забираем все данные прям из preview
    for block in div_blocks:
        if block.find('a', class_= 'tm-article-snippet__readmore'):
            href_list.append(block.find('a', class_= 'tm-article-snippet__readmore').get('href'))
        else:
            date = pd.to_datetime(block.find('div', class_ = 'tm-article-snippet__meta').find('time').get('datetime')).date()
            title = block.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
            text = block.find('div', class_= 'article-formatted-body article-formatted-body_version-1').text
            row = {'date':date, 'title':title, 'url':url+'/?q=python, парсинг', 'text':text}
            all_post = pd.concat([all_post, pd.DataFrame([row])])
all_links = all_links + list(map(lambda x: "https://habr.com" + x, href_list))
# в цикле заходим по ссылкам на полный текст и вытаскиваем все необходимые данные
for link in all_links:
    soup = BeautifulSoup(req.get(link).text)
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('div', class_ = 'tm-article-snippet__meta').find('time').get('datetime')).date()
    if soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1'):
        title = soup.find('h1', class_='tm-article-snippet__title tm-article-snippet__title_h1').text
    else:            
        title = soup.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2').text
    text = soup.find('div', xmlns="http://www.w3.org/1999/xhtml").text
    row = {'date':date, 'title':title, 'url':link, 'text':text}
    all_post = pd.concat([all_post, pd.DataFrame([row])])
all_post







Wall time: 14min 11s


,date,title,url,text
0,2018-08-16,10 самых распространенных ошибок безопасности ...,"https://habr.com/ru/search/page3/?q=python, па...",Всем привет!\n\r\nНаша очередная группа по Pyt...
0,2020-01-23,"А что, если без Python? Julia для машинного об...","https://habr.com/ru/search/page5/?q=python, па...","Мы всегда хотим писать код быстро, но за это п..."
0,2018-10-11,Открытый вебинар «Как не нужно писать на Python»,"https://habr.com/ru/search/page6/?q=python, па...",Всем привет! В рамках нашего курса «Разработчи...
0,2019-01-08,Удалённое управление эмулятором Fceux с помощь...,"https://habr.com/ru/search/page7/?q=python, па...","В статье я опишу, как сделать эмулятор NES упр..."
0,2021-06-25,Улучшение освещения первого Quake трассировкой...,"https://habr.com/ru/search/page8/?q=python, па...","\nСпидран первого эпизода Quake, отрендеренный..."
...,...,...,...,...
0,2021-08-10,HackTheBox endgame. Прохождение лаборатории RP...,https://habr.com/ru/post/572158/,"\nПривет, с вами Ральф! В данной статье разбер..."
0,2021-08-13,Как мы автоматизировали процесс генерации Rele...,https://habr.com/ru/company/ru_mts/blog/572774/,Всем привет! Меня зовут Семен. Я Java-разработ...
0,2021-08-26,Как мы добавили поддержку языка Frege в IDEA. ...,https://habr.com/ru/company/hsespb/blog/574692/,"Привет! В этом посте мы расскажем, как реализо..."
0,2021-09-09,FOSS News №87 – Спецвыпуск про Telegram бота: ...,https://habr.com/ru/post/577178/,Дайджесты FOSS News мы выпускаем с конца январ...


# Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

In [3]:
#Формируем параметры для запроса
import requests as req  
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {'emailAddresses':['maria70@yandex.ru']}
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Content-Length': '40',
    'Content-Type': 'application/json;charset=utf-8',
    'Connection': 'keep-alive',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Version': '0'
}
res = req.post(url, headers = headers, json = params)
res.status_code
res.content

b'null'